# Building a Retrieval-Augmented Generation (RAG) Application with LangChain and MistralAI

This notebook demonstrates how to build a RAG application using LangChain and MistralAI to query over a dataset of markdown documents.

## 1. Setup and Imports

Import the necessary libraries and load your API keys.

In [9]:
import os
from dotenv import load_dotenv
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_mistralai.chat_models import ChatMistralAI

# Load environment variables
load_dotenv()
mistral_api_key = os.getenv("MISTRAL_API_KEY")

# Ensure the API key is loaded
assert mistral_api_key is not None, "Please set your MISTRAL_API_KEY in the .env file."

## 2. Load Markdown Documents

Load all markdown files from the `data/` directory, including all subdirectories.

In [12]:
import os

def load_documents(root_dir):
    documents = []
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith(".md"):
                filepath = os.path.join(dirpath, filename)
                loader = UnstructuredMarkdownLoader(filepath)
                documents.extend(loader.load())
    return documents

# Load documents from the data directory
docs = load_documents("data")

print(f"Loaded {len(docs)} documents.")

Loaded 1156 documents.


## 3. Split Documents into Chunks

Split the documents into smaller chunks to improve retrieval efficiency.

In [13]:
# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""]
)

# Split the documents
split_docs = text_splitter.split_documents(docs)

print(f"Split into {len(split_docs)} chunks.")

Split into 12733 chunks.


## 4. Create Embeddings for the Text Chunks

Convert the text chunks into vector embeddings using a pre-trained model.

In [14]:
# Initialize the embedding model
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/tmp/ipykernel_258562/1469870572.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/home/benjamin/Documents/IA/telecom-rag/env/lib64/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


## 5. Build a Vector Store with FAISS

Store the embeddings in a vector store to enable efficient similarity searches.

In [15]:
# Build the FAISS vector store from the documents and embeddings
vectorstore = FAISS.from_documents(split_docs, embedding)

print("Vector store created.")

Vector store created.


## 6. Set Up a Retriever

Create a retriever that uses the vector store to find relevant documents based on a query.

In [22]:
# Create a retriever from the vector store
retriever = vectorstore.as_retriever()

# Optional: Customize retriever settings
retriever.search_kwargs['k'] = 50  # Number of documents to retrieve

## 7. Initialize the MistralAI Language Model

Set up the MistralAI language model to generate responses.

In [23]:
# Initialize the MistralAI language model
llm = ChatMistralAI(model="mistral-small", api_key=mistral_api_key)

## 8. Create a RetrievalQA Chain

Combine the retriever and the language model into a chain that can handle queries.

In [24]:
# Create the RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)

print("RetrievalQA chain created.")

RetrievalQA chain created.


## 9. Test the RAG Application

Run a test query to see the RAG application in action.

In [28]:
# Define a test query
query = "Quelles sont les UEs de la filières IGR ? Qui est le responsable de la filière ?"

# Run the query through the RetrievalQA chain
result = qa_chain.run(query)

# Display the result
print("Query :", query)
print("")
print("Answer:", result)

Query : Donnes moi la liste des filières de 2ème année.

Answer: The list of 2nd year specializations includes:

1. Economics and Digital Innovation
2. Markets, Organization, Data, Strategy
3. Artificial Intelligence and Data Science
4. Image
5. Data Science
6. Signal Processing for Artificial Intelligence
7. Applied Algebra
8. Mathematics, Theoretical Computer Science and Operational Research
9. Random Modeling and Scientific Computing
10. Networks, Communications and Cybersecurity
11. Cybersecurity
12. Large Digital Infrastructures
13. Mobile Networks and Internet of Things

These specializations are composed of units of teaching (UE) in 2nd year, which open on options of 3rd year. The courses are 24 hours or 48 hours each. Students must choose 2 specializations among 14.


## 10. Conclusion

You've successfully built a RAG application that can answer queries based on your markdown documents. You can now expand upon this foundation to handle more complex queries or integrate additional features.